In [1]:
import tomotopy as tp
print(tp.isa)

avx2


In [2]:
# Read uspto dataset
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

usptodata = pd.read_csv('U.S. Patents.csv')
usptodataset=usptodata[["grant_id","claims_text","abstract"]]
usptodataset= usptodataset.dropna()
usptodataset = usptodataset.reset_index(drop=True)
usptodataset.head()

,grant_id,claims_text,abstract
0,USPP030977,1. A new and distinct Mango plant characterize...,"A new and distinct variety of Mango plant, her..."
1,USPP030978,1. A new and distinct apple tree substantially...,&#x2018;Honeysuckle Rose #1-6&#x2019; is a new...
2,USPP030979,1. A new and distinct variety of peach tree as...,"A new and distinct peach tree variety, <i>Prun..."
3,USPP030980,1. A new and distinct variety of raspberry pla...,This invention relates to a new and distinct v...
4,USPP030981,1. A new and distinct Strawberry plant named &...,A new and distinct cultivar of Strawberry plan...


In [3]:
#Clean and token data
stopwords = nltk.corpus.stopwords.words('english')

def lemma(word): #map pos tag
    tag = nltk.pos_tag([word])[0][1][0].upper()
    allowed_tags = {'J','N', 'V', 'R'}
    if tag in allowed_tags:
        lemma = WordNetLemmatizer().lemmatize(word)
        return lemma
    else:
        return ''
    
def  prepare_text(text):
    tokens = word_tokenize(text)
    tokens = [token  for token in tokens if len(token)> 3]
    tokens = [token for token in tokens if token not in stopwords]
    token = [lemma(token) for token in tokens]
    return tokens
    
usptodataset['Text'] = usptodataset['claims_text'].astype(str)
usptodataset['Text'] = usptodataset['Text'].apply(prepare_text)


In [4]:
usptodataset.head()

,grant_id,claims_text,abstract,Text
0,USPP030977,1. A new and distinct Mango plant characterize...,"A new and distinct variety of Mango plant, her...","[distinct, Mango, plant, characterized, follow..."
1,USPP030978,1. A new and distinct apple tree substantially...,&#x2018;Honeysuckle Rose #1-6&#x2019; is a new...,"[distinct, apple, tree, substantially, describ..."
2,USPP030979,1. A new and distinct variety of peach tree as...,"A new and distinct peach tree variety, <i>Prun...","[distinct, variety, peach, tree, herein, descr..."
3,USPP030980,1. A new and distinct variety of raspberry pla...,This invention relates to a new and distinct v...,"[distinct, variety, raspberry, plant, illustra..."
4,USPP030981,1. A new and distinct Strawberry plant named &...,A new and distinct cultivar of Strawberry plan...,"[distinct, Strawberry, plant, named, x2018, P0..."


In [5]:
#split data to have little data to run
# divide dataset to train and test
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(usptodataset, test_size=0.8, random_state=25)
df_train=df_train.reset_index(drop=True)
df_train

,grant_id,claims_text,abstract,Text
0,US10459019,"1. An electromagnetic sensor comprising:,a fir...",An electromagnetic sensor includes a first mag...,"[electromagnetic, sensor, comprising, first, d..."
1,US10456083,1. A method for mapping somatosensory and moto...,An apparatus for cortical mapping and method f...,"[method, mapping, somatosensory, motor, cortic..."
2,US10461549,"1. A method for charging a mobile terminal, th...","The disclosure discloses a mobile terminal, a ...","[method, charging, mobile, terminal, method, c..."
3,US10462815,1. A method for a User Equipment (UE) operatin...,The present invention relates to a wireless co...,"[method, User, Equipment, operating, wireless,..."
4,US10458026,1. A method of producing graphene sheets compr...,A method of producing graphene sheets comprisi...,"[method, producing, graphene, sheets, comprisi..."
...,...,...,...,...
1397,US10458777,1. A method of measuring a metrology target el...,"Targets, target elements and target design met...","[method, measuring, metrology, target, element..."
1398,US10458022,1. A method for anti-corrosive treatment of me...,"A method for corrosion protection treatment, c...","[method, anti-corrosive, treatment, metallic, ..."
1399,US10462550,"1. A storage device comprising:,a first case c...","A storage device includes a first case, a seco...","[storage, device, comprising, first, case, com..."
1400,US10456037,1. A terminal device configured to be able to ...,A terminal device is provided which is configu...,"[terminal, device, configured, able, communica..."


In [6]:
def cleantext(df): 
    
    df_train['cleaned_text'] = df_train['claims_text'].replace(r'\'|\"|\,|\.|\?|\+|\-|\/|\=|\(|\)|\n|"', '', regex=True)
    df_train['cleaned_text'] = df_train['cleaned_text'].replace("  ", " ")
    
    # convert tweets to lowercase
    df_train['cleaned_text'] = df_train['cleaned_text'].str.lower()
    
    #remove numbers
    df_train['cleaned_text'] =df_train['cleaned_text'].replace(r'\d+', '', regex = True)
        
    #remove_symbols
    df_train['cleaned_text']  = df_train['cleaned_text'].replace(r'[^a-zA-Z0-9]', " ", regex=True)
    
    #remove punctuations 
    df_train['cleaned_text'] = df_train['cleaned_text'].replace(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+',"", regex = True)
    
    #remove_URL(x):
    df_train['cleaned_text']  = df_train['cleaned_text'].replace(r'https.*$', "", regex = True)
    
    #remove stopwords and words_to_remove
    df_train['fully_cleaned_text'] = df_train['cleaned_text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stopwords]))
    
    return df_train

df = cleantext(df_train)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\inference.py:177: FutureWarning: Possible nested set at position 1
  re.compile(obj)


In [7]:
df 

,grant_id,claims_text,abstract,Text,cleaned_text,fully_cleaned_text
0,US10459019,"1. An electromagnetic sensor comprising:,a fir...",An electromagnetic sensor includes a first mag...,"[electromagnetic, sensor, comprising, first, d...",an electromagnetic sensor comprising a first ...,electromagnetic sensor comprising first diaphr...
1,US10456083,1. A method for mapping somatosensory and moto...,An apparatus for cortical mapping and method f...,"[method, mapping, somatosensory, motor, cortic...",a method for mapping somatosensory and motor ...,method mapping somatosensory motor cortical re...
2,US10461549,"1. A method for charging a mobile terminal, th...","The disclosure discloses a mobile terminal, a ...","[method, charging, mobile, terminal, method, c...",a method for charging a mobile terminal the m...,method charging mobile terminal method compris...
3,US10462815,1. A method for a User Equipment (UE) operatin...,The present invention relates to a wireless co...,"[method, User, Equipment, operating, wireless,...",a method for a user equipment ue operating in...,method user equipment ue operating wireless co...
4,US10458026,1. A method of producing graphene sheets compr...,A method of producing graphene sheets comprisi...,"[method, producing, graphene, sheets, comprisi...",a method of producing graphene sheets compris...,method producing graphene sheets comprising st...
...,...,...,...,...,...,...
1397,US10458777,1. A method of measuring a metrology target el...,"Targets, target elements and target design met...","[method, measuring, metrology, target, element...",a method of measuring a metrology target elem...,method measuring metrology target element comp...
1398,US10458022,1. A method for anti-corrosive treatment of me...,"A method for corrosion protection treatment, c...","[method, anti-corrosive, treatment, metallic, ...",a method for anticorrosive treatment of metal...,method anticorrosive treatment metallic surfac...
1399,US10462550,"1. A storage device comprising:,a first case c...","A storage device includes a first case, a seco...","[storage, device, comprising, first, case, com...",a storage device comprising a first case comp...,storage device comprising first case comprisin...
1400,US10456037,1. A terminal device configured to be able to ...,A terminal device is provided which is configu...,"[terminal, device, configured, able, communica...",a terminal device configured to be able to co...,terminal device configured able communicate se...


In [8]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

df['fully_cleaned_text'] = df['fully_cleaned_text'].apply(lemmatize_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
df['fully_cleaned_text']

0       electromagnetic sensor comprise first diaphrag...
1       method map somatosensory motor cortical region...
2       method charge mobile terminal method comprise ...
3       method user equipment ue operate wireless comm...
4       method produce graphene sheet comprise step of...
                              ...                        
1397    method measure metrology target element compri...
1398    method anticorrosive treatment metallic surfac...
1399    storage device comprise first case comprise fi...
1400    terminal device configure able communicate ser...
1401    organic lightemitting display device comprise ...
Name: fully_cleaned_text, Length: 1402, dtype: object

In [10]:
#save dataframe to txt file
df['fully_cleaned_text'].to_csv(r'C:\Users\admin\Topic-Modeling\U.S.Patents.txt', header=None, index=None, sep=' ', mode='a')

In [11]:
import tomotopy as tp
#class HLDAModel(tw=TermWeight.ONE, min_cf=0, min_df=0, rm_top=0, depth=2, alpha=0.1, eta=0.01, gamma=0.1, seed=None, corpus=None, transform=None)


file_name = 'U.S.Patents.txt'

mdl = tp.HLDAModel(depth=3)     
for line in open(file_name, 'r'):
    document = line.strip().split()
    mdl.add_doc(document)

print('Training model by iterating over the corpus 50 times, 10 iterations at a time')
iterations = 10
for i in range(0, 50, iterations):
    mdl.train(iterations)
    print('Iteration: #{}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    if not mdl.is_live_topic(k):
        continue
    print('child of topic #{} - Level: #{}, number of documents:#{}'.format(mdl.parent_topic(k), mdl.level(k), mdl.num_docs_of_topic(k)) )
    print('Top 10 words of global topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=5))

Training model by iterating over the corpus 50 times, 10 iterations at a time
Iteration: #0	Log-likelihood: -4.886112320297857
Iteration: #10	Log-likelihood: -4.782665144613714
Iteration: #20	Log-likelihood: -4.742281088859614
Iteration: #30	Log-likelihood: -4.720962680241537
Iteration: #40	Log-likelihood: -4.708134569860517
child of topic #-1 - Level: #0, number of documents:#5608
Top 10 words of global topic #0
[('first', 0.3229125142097473), ('second', 0.25890272855758667), ('wherein', 0.08997419476509094), ('claim', 0.07169947028160095), ('one', 0.06076038256287575)]
child of topic #0 - Level: #1, number of documents:#164
Top 10 words of global topic #8
[('one', 0.040975362062454224), ('least', 0.03193700686097145), ('electrode', 0.022296102717518806), ('wherein', 0.019885875284671783), ('electrodes', 0.019283318892121315)]
child of topic #0 - Level: #1, number of documents:#8
Top 10 words of global topic #9
[('first', 5.8011373766930774e-05), ('wherein', 5.8011373766930774e-05), (

In [13]:
mdl.summary()

<Basic Info>
| HLDAModel (current version: 0.12.3)
| 5608 docs, 3542948 words
| Total Vocabs: 17238, Used Vocabs: 17238
| Entropy of words: 7.09163
| Entropy of term-weighted words: 7.09163
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 50, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -4.70813
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| depth: 3 (the maximum depth level of hierarchy between 2 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-depth level, given as a single `float` in case of symmetric prior and as a list with length `depth` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| gamma: 0.1 (concentration coeficient of Dirichlet Process)
| seed: 533486436 (random seed)
| trained in version 0.12.3
|


In [ ]:
# import tomotopy as tp
# file_name = 'U.S.Patents.txt'

# def tokenizer(x): return x.split()
# corpus = [tokenizer(s) for s in file_name]


# hLDA_tomotopy_model = tp.HLDAModel(depth=3, min_df=1, rm_top=100)
# for token_list in corpus: hLDA_tomotopy_model.add_doc(token_list)
# for i in range(0,1000,100): hLDA_tomotopy_model.train(100)
# hLDA_tomotopy_model.summary()  
  

In [ ]:
# # evaluation
# import gensim
# from gensim import corpora
# from gensim.models import CoherenceModel

# text_data = df['Text']
# dictionary = corpora.Dictionary(text_data)

In [ ]:
# hlda=mdl.vocabs
# npmi_Coherence_hLDA = CoherenceModel(model=hlda , texts=text_data, dictionary= dictionary, coherence='c_npmi') 
# print("npmi_Coherence for hdp: " + str(npmi_Coherence_hLDA.get_coherence()))

# cv_Coherence_hlda = CoherenceModel(model=hlda, texts=text_data, dictionary= dictionary, coherence='c_v') 
# print("cv_Coherence for hdp: "+ str(cv_Coherence_hlda.get_coherence()))

In [15]:
import tomotopy as tp
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))  
corpus = tp.utils.Corpus(tokenizer=tp.utils.SimpleTokenizer(stemmer=stemmer.stem), 
    stopwords=lambda x: len(x) <= 2 or x in stopwords)
corpus.process(open('U.S.Patents.txt', encoding='utf-8'))

5608

In [16]:
targets = mdl.vocabs
npmi_Coherence =tp.coherence.Coherence(corpus, coherence='c_npmi', window_size=5, targets=targets, top_n=10, eps=1e-12, gamma=1.0)
